In [4]:
import praw
import os
from dotenv import load_dotenv, dotenv_values 
import json
from datetime import datetime

In [5]:
#Inicializamos las credenciales para el uso API
load_dotenv()
CLIENT_ID = os.getenv('CLIENT_ID')
SECRET_KEY = os.getenv('SECRET_KEY')
USER_AGENT = os.getenv('USER_AGENT')
USERNAMEAPI = os.getenv('USERNAMEAPI')
PASSWORD = os.getenv('PASSWORD')

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=SECRET_KEY,
    user_agent=USER_AGENT,
    username=USERNAMEAPI,
    password=PASSWORD
)

In [6]:
import datetime
import json

start_date = datetime.datetime(2023, 10, 1)
end_date = datetime.datetime(2024, 10, 1)

start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

subreddit_list = []

subreddits = reddit.subreddits.popular(limit=100)

for subreddit in subreddits:
    subreddit_data = {
        "tittleSubreddit": subreddit.display_name,
        "subscribersReddit": subreddit.subscribers,
        "descriptionReddit": subreddit.public_description,
        "createdReddit": subreddit.created_utc,
        "subreddit_url": f"https://www.reddit.com{subreddit.url}",
        "upVotedPosts": []
    }

    # Obtener las publicaciones más populares del último año
    for submission in subreddit.top(limit=2000, time_filter="year"):
        # Filtrar por fecha de publicación dentro del rango deseado
        if start_timestamp <= submission.created_utc <= end_timestamp:
            submission_data = {
                "author": str(submission.author),
                "titlePost": submission.title,
                "selftext": submission.selftext,
                "createdPost": submission.created_utc,
                "scorePost": submission.score,
                "upVotedRatio": submission.upvote_ratio,
                "upVotes": submission.ups,
                "flair": submission.link_flair_text,
                "commentsPost": submission.num_comments,
                "urlPost": submission.url
            }

            subreddit_data["upVotedPosts"].append(submission_data)

    # Agregar datos del subreddit a la lista
    subreddit_list.append(subreddit_data)

# Guardar los resultados en un archivo JSON
with open('subreddits.json', 'w') as json_file:
    json.dump(subreddit_list, json_file, indent=4)
